In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.5
SparkUI available at http://hail-m.us-central1-f.c.cncd-cncd.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /hail-20210211-1007-0.2.61-3c86d3ba497a.log


In [2]:
files = ["gs://pgr-wes-r20/GL_by_chrom/CNCD_Freeze_Two.6.GL.vcf.gz", "gs://pgr_wes_30k/GRCh38/chr6_30k_grch38.vcf.bgz",
        "gs://pgr-wgs-8k/VCF/vcf_DP10/pgr7k_chr6.vcf.gz", "gs://pgr-wgs-2k/chr6_PAK_indv.recode.vcf.gz"]

recode = {f"{i}":f"chr{i}" for i in (list(range(1, 23)) + ['X', 'Y'])}

pos = 'chr6:3105480-3105481'

for file in files:
    data = hl.import_vcf(file, reference_genome="GRCh38", force_bgz=True, contig_recoding=recode)
    ds_result1 = hl.filter_intervals(data, [hl.parse_locus_interval(pos, reference_genome='GRCh38')])
    ds_result1.rows().show()
    if ds_result1.count_rows() != 0:
        print('Annotate Heterozygous Alleles')
        ds_result1 = ds_result1.annotate_rows(hets = hl.agg.filter(ds_result1.GT.is_het(),hl.agg.collect(ds_result1.s)))
        ds_result1.count()
        print('Annotate N Hets')
        ds_result4 = ds_result1.annotate_rows(n_het = hl.agg.count_where(ds_result1.GT.is_het()))
        ds_result4.count()
        print('Annotate Homozygous Alleles')
        ds_result3 = ds_result4.annotate_rows(homs = hl.agg.filter(ds_result4.GT.is_hom_var(),hl.agg.collect(ds_result4.s)))
        ds_result3.count()
        print('Annotating N Homs')
        ds_result5 = ds_result3.annotate_rows(n_hom = hl.agg.count_where(ds_result3.GT.is_hom_var()))

        ds_result5.rows().export("gs://test1haseeb/Kashif-Lookup/"+file+pos+".tsv")
    

2021-02-11 10:08:27 Hail: INFO: Coerced sorted dataset
2021-02-11 10:08:27 Hail: INFO: reading 1 of 79 data partitions


+---------------+------------+-----------------+----------+----------+
| locus         | alleles    | rsid            |     qual | filters  |
+---------------+------------+-----------------+----------+----------+
| locus<GRCh38> | array<str> | str             |  float64 | set<str> |
+---------------+------------+-----------------+----------+----------+
| chr6:3105480  | ["A","T"]  | "6_3105480_A_T" | 2.07e+02 | NA       |
+---------------+------------+-----------------+----------+----------+

+----------------+--------------+
| info.AF        | info.AQ      |
+----------------+--------------+
| array<float64> | array<int32> |
+----------------+--------------+
| [2.60e-05]     | [207]        |
+----------------+--------------+

2021-02-11 10:08:51 Hail: INFO: Coerced sorted dataset
2021-02-11 10:08:51 Hail: INFO: reading 1 of 79 data partitions


Annotate Heterozygous Alleles


2021-02-11 10:09:11 Hail: INFO: Coerced sorted dataset
2021-02-11 10:09:11 Hail: INFO: reading 1 of 79 data partitions


...

2021-02-11 10:25:49 Hail: INFO: Coerced sorted dataset
2021-02-11 10:25:49 Hail: INFO: reading 1 of 199 data partitions


Annotating N Homs


2021-02-11 10:27:24 Hail: INFO: Coerced sorted dataset
2021-02-11 10:27:24 Hail: INFO: reading 1 of 199 data partitions
2021-02-11 10:32:39 Hail: INFO: merging 1 files totalling 1.2K...
2021-02-11 10:32:39 Hail: INFO: while writing:
    gs://test1haseeb/Kashif-Lookup/gs://pgr-wgs-8k/VCF/vcf_DP10/pgr7k_chr6.vcf.gzchr6:3105480-3105481.tsv
  merge time: 387.171ms
2021-02-11 10:36:07 Hail: INFO: Coerced sorted dataset
2021-02-11 10:36:07 Hail: INFO: reading 1 of 707 data partitions


+---------------+------------+------+---------+----------+--------------+
| locus         | alleles    | rsid |    qual | filters  | info.AC      |
+---------------+------------+------+---------+----------+--------------+
| locus<GRCh38> | array<str> | str  | float64 | set<str> | array<int32> |
+---------------+------------+------+---------+----------+--------------+
+---------------+------------+------+---------+----------+--------------+

+----------------+---------+-------------------+----------------------+
| info.AF        | info.AN | info.BaseQRankSum | info.ClippingRankSum |
+----------------+---------+-------------------+----------------------+
| array<float64> |   int32 |           float64 |              float64 |
+----------------+---------+-------------------+----------------------+
+----------------+---------+-------------------+----------------------+

+---------+---------+---------+----------+---------+---------------------+
| info.DB | info.DP | info.DS | info.END | info.FS | info.HaplotypeScore |
+---------+---------+---------+----------+---------+---------------------+
|    bool |   int32 |    bool |    int32 | float64 |             float64 |
+---------+---------+---------+----------+---------+---------------------+
+---------+---------+---------+----------+---------+---------------------+

+----------------------+--------------+----------------+---------+----------+
| info.InbreedingCoeff | info.MLEAC   | info.MLEAF     | info.MQ | info.MQ0 |
+----------------------+--------------+----------------+---------+----------+
|              float64 | array<int32> | array<float64> | float64 |    int32 |
+----------------------+--------------+----------------+---------+----------+
+----------------------+--------------+----------------+---------+----------+

+----------------+--------------------------+--------------------------+
| info.MQRankSum | info.NEGATIVE_TRAIN_SITE | info.POSITIVE_TRAIN_SITE |
+----------------+--------------------------+--------------------------+
|        float64 |                     bool |                     bool |
+----------------+--------------------------+--------------------------+
+----------------+--------------------------+--------------------------+

+---------+---------------------+----------+-------------+--------------+
| info.QD | info.ReadPosRankSum | info.SOR | info.VQSLOD | info.culprit |
+---------+---------------------+----------+-------------+--------------+
| float64 |             float64 |  float64 |     float64 | str          |
+---------+---------------------+----------+-------------+--------------+
+---------+---------------------+----------+-------------+--------------+

2021-02-11 10:37:13 Hail: INFO: Coerced sorted dataset
2021-02-11 10:37:13 Hail: INFO: reading 1 of 707 data partitions


In [ ]:
ds_result1.count()

In [ ]:
# if ds_result1.count()[0] != 0:
ds_result1.count_rows()

In [ ]:
data6 = hl.read_matrix_table("gs://test1haseeb/GOLGA6L6_20K_vep.mt")
data6.describe()

In [ ]:
data6 = hl.read_matrix_table("gs://test1haseeb/GOLGA6L6_20K_vep.mt")
data1 = data6.annotate_rows(
    info=data6.info.annotate(GeneName=data6.vep.transcript_consequences.gene_symbol,
                             EnsembleGeneId=data6.vep.transcript_consequences.gene_id,
                             variantConsequence=data6.vep.most_severe_consequence,
                             consequence_terms=data6.vep.transcript_consequences.consequence_terms,
                             transcript_affected=data6.vep.transcript_consequences.transcript_id,
                             lof=data6.vep.transcript_consequences.lof,
                             lof_filter=data6.vep.transcript_consequences.lof_filter,
                             biotype=data6.vep.transcript_consequences.biotype,
                             canonical=data6.vep.transcript_consequences.canonical
                             # should we add csq_score????
                               ))
data1.rows().show()
data2 = data1.rows().to_pandas()
data2.to_excel('GOLGA6L6_20K_vep.xlsx')
!gsutil cp GOLGA6L6_20K_vep.xlsx gs://test1haseeb/var-lookup/

In [ ]:
data6 = hl.read_matrix_table("gs://test1haseeb/GALNT9_30K_vep.mt")
data1 = data6.annotate_rows(
    info=data6.info.annotate(GeneName=data6.vep.transcript_consequences.gene_symbol,
                             EnsembleGeneId=data6.vep.transcript_consequences.gene_id,
                             variantConsequence=data6.vep.most_severe_consequence,
                             consequence_terms=data6.vep.transcript_consequences.consequence_terms,
                             transcript_affected=data6.vep.transcript_consequences.transcript_id,
                             lof=data6.vep.transcript_consequences.lof,
                             lof_filter=data6.vep.transcript_consequences.lof_filter,
                             biotype=data6.vep.transcript_consequences.biotype,
                             canonical=data6.vep.transcript_consequences.canonical
                             # should we add csq_score????
                               ))
data1.rows().show()
data2 = data1.rows().to_pandas()
data2.to_excel('GALNT9_30K_vep.xlsx')
!gsutil cp GALNT9_30K_vep.xlsx gs://test1haseeb/var-lookup/
# data1.rows().export("gs://test1haseeb/var-lookup/GALNT9_30K_vep.tsv")

In [ ]:
data6 = hl.read_matrix_table("gs://test1haseeb/KIR3DL1_20K_vep.mt")
data1 = data6.annotate_rows(
    info=data6.info.annotate(GeneName=data6.vep.transcript_consequences.gene_symbol,
                             EnsembleGeneId=data6.vep.transcript_consequences.gene_id,
                             variantConsequence=data6.vep.most_severe_consequence,
                             consequence_terms=data6.vep.transcript_consequences.consequence_terms,
                             transcript_affected=data6.vep.transcript_consequences.transcript_id,
                             lof=data6.vep.transcript_consequences.lof,
                             lof_filter=data6.vep.transcript_consequences.lof_filter,
                             biotype=data6.vep.transcript_consequences.biotype,
                             canonical=data6.vep.transcript_consequences.canonical
                             # should we add csq_score????
                               ))
data1.rows().show()
data2 = data1.rows().to_pandas()
data2.to_excel('KIR3DL1_20K_vep.xlsx')
!gsutil cp KIR3DL1_20K_vep.xlsx gs://test1haseeb/var-lookup/
# data1.rows().export("gs://test1haseeb/var-lookup/KIR3DL1_20K_vep.tsv")

In [ ]:
data6 = hl.read_matrix_table("gs://test1haseeb/LYPLAL1_20K.mt")
data1 = data6.annotate_rows(
    info=data6.info.annotate(GeneName=data6.vep.transcript_consequences.gene_symbol,
                             EnsembleGeneId=data6.vep.transcript_consequences.gene_id,
                             variantConsequence=data6.vep.most_severe_consequence,
                             consequence_terms=data6.vep.transcript_consequences.consequence_terms,
                             transcript_affected=data6.vep.transcript_consequences.transcript_id,
                             lof=data6.vep.transcript_consequences.lof,
                             lof_filter=data6.vep.transcript_consequences.lof_filter,
                             biotype=data6.vep.transcript_consequences.biotype,
                             canonical=data6.vep.transcript_consequences.canonical
                             # should we add csq_score????
                               ))
data1.rows().show()
data2 = data1.rows().to_pandas()
data2.to_excel('LYPLAL1_20K_vep.xlsx')
!gsutil cp LYPLAL1_20K_vep.xlsx gs://test1haseeb/var-lookup/
# data1.rows().export("gs://test1haseeb/var-lookup/LYPLAL1_20K_vep.tsv")

In [ ]:
data6 = hl.read_matrix_table("gs://test1haseeb/LYPLAL1_30K_vep.mt")
data1 = data6.annotate_rows(
    info=data6.info.annotate(GeneName=data6.vep.transcript_consequences.gene_symbol,
                             EnsembleGeneId=data6.vep.transcript_consequences.gene_id,
                             variantConsequence=data6.vep.most_severe_consequence,
                             consequence_terms=data6.vep.transcript_consequences.consequence_terms,
                             transcript_affected=data6.vep.transcript_consequences.transcript_id,
                             lof=data6.vep.transcript_consequences.lof,
                             lof_filter=data6.vep.transcript_consequences.lof_filter,
                             biotype=data6.vep.transcript_consequences.biotype,
                             canonical=data6.vep.transcript_consequences.canonical
                             # should we add csq_score????
                               ))
data1.rows().show()
data2 = data1.rows().to_pandas()
data2.to_excel('LYPLAL1_30K_vep.xlsx')
!gsutil cp LYPLAL1_30K_vep.xlsx gs://test1haseeb/var-lookup/
# data1.rows().export("gs://test1haseeb/var-lookup/LYPLAL1_30K_vep.tsv")